Get the difference between nucleus and TFS set of words. See if the words in this set are reasonable. They should be reasonable for when TFS is looser and should not be when TFS is tighter. Out of this set of words, take the highest, lowest prob ones and the middle one? Need at least 3 word gap between them. 
Rank each one in how reasonable it is. Control baseline is how reasonable words at different points in the body are. The first word, half way and at the closer of the tails. Can also get words outside of the last tail, just outside, halfway to the end, and at the very end. 
There are 9 different words in total. Should i rank all of them, scramble up a subset? Or have a ranking for each of the subsets differently? Probably scramble them all up. And then present 3 of them??
Can then see overall how replaceable different parts of the distributions is, and for TFS and nucleus specifically. Have a control with the bad words. And should be able to hopefully show that the difference in replaceability between the TFS and nucleus words is small when TFS is looser and large when TFS is tighter. Can see how specific this is also. Eg could just look at the last TFS and nucleus words, see the diff in replaceability, or maybe need to look at the averages. Look most at this unique subset and these three points should hopefully categorize it. Want them because i dont know how good the signal is going to be. For the closest words for example. 
Can also get overall data on how replaceability corresponds to model probability. 


In [13]:
import pickle
import gzip
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
import pandas as pd    
import torch

In [54]:
from decodeLogits import *

In [14]:
#updated
vals_dict = {'tfs':[0.25, 0.75, 0.9, 0.95, 0.99],
'n': [0.5, 0.63, 0.69, 0.81, 0.75, 0.9], 'k':[1,40,200]  }


''' {'tfs':[0.25, 0.75, 0.9, 0.95, 0.99], 'flat':[0.01, 0.02, 0.05],
'n': [0.1, 0.25, 0.5, 0.63, 0.69, 0.75, 0.81, 0.9], 'k':[1,10,40,200]  }'''

" {'tfs':[0.25, 0.75, 0.9, 0.95, 0.99], 'flat':[0.01, 0.02, 0.05],\n'n': [0.1, 0.25, 0.5, 0.63, 0.69, 0.75, 0.81, 0.9], 'k':[1,10,40,200]  }"

# Doing analysis for the same logit going to use the ground truth as it is a Schelling point

In [15]:
from importlib import reload
import utils

reload(utils)
from utils import *

In [16]:
batch_size=25
num_batches=4
prompt_length=100
generated_length=150
tot_len = prompt_length+generated_length

import encoder
model_name='345M'
models_dir='../gpt-2/models'
enc = encoder.get_encoder(model_name, models_dir)

prompts=pd.read_csv('test_dataframe_500primer.csv')

In [17]:
gpt_out_path = 'gpt-2_output/'
additional_path = '-model_774M-seed_27'
#all_perps = pickle.load( gzip.open(gpt_out_path+'all_perplexities_perplexity_scores_for_the_dataset_Human_StoryPrompts_Completion.csv'+additional_path+'.pickle.gz', 'rb'))
all_logits = pickle.load( gzip.open(gpt_out_path+'all_logits_perplexity_scores_for_the_dataset_Human_StoryPrompts_Completion.csv'+additional_path+'.pickle.gz', 'rb')) # needed to get the probabilities
text = pickle.load( gzip.open(gpt_out_path+'all_text_perplexity_scores_for_the_dataset_Human_StoryPrompts_Completion.csv'+additional_path+'.pickle.gz', 'rb'))

In [160]:
tot_num = 0
mapping_to_batch = dict()
for i in range(num_batches):
    num_in_batch = all_logits[i].shape[0]
    for el, b_ind in zip(range(tot_num, tot_num+num_in_batch), range(0, num_in_batch)):
        mapping_to_batch[el] = (i, b_ind) # actual batch and then the ind in that batch
    print( num_in_batch)
    tot_num+= num_in_batch
tot_num

23
23
24
22


92

In [167]:
all_logits[0].shape

(23, 150, 50257)

## Want to also focus on the maximum deviation portions at some point. 

In [169]:
# random prompts and locations within the 150 generation locations
num_prompts_and_timepoints_wanted = 10
leading_prompt = 15

rand_prompts = np.random.randint(0,tot_num, num_prompts_and_timepoints_wanted)
# leadingprompt until 150. as the logits are 150 not 250. 
# -1 because of the end token prediction
rand_times = np.random.randint(0,generated_length-1, num_prompts_and_timepoints_wanted)

In [ ]:
# selecting randomly from the max deviations: 



In [170]:
from importlib import reload
import utils

reload(utils)
from utils import *

In [171]:
len(all_logits[0][0, :, 0])

150

In [172]:
lol = [1,2,3,4,5]
lol[:2]

[1, 2]

getting random locations
for the differnet gen strategies. find the tail locations. take the tokens at each position. 
move onto the next random location. 

In [175]:
# applying all analyses to the logits. 
prompt_length = 100
methods_wanted = ['tfs_0.95', 'n_0.69']#, 'k_40']

to_df = []

# randomly select generations and positions in those generations. from the original prompts. 
for r_ind in range(0, num_prompts_and_timepoints_wanted):
    
    prompt = rand_prompts[r_ind]
    time_point = rand_times[r_ind]
    text_timepoint = time_point+prompt_length+1 # +1 because the perplexities keeps the end prediction token. 
    batch_ind, ind_in_batch = mapping_to_batch[prompt]
    
    diff_tail_positions = dict()
    largest_tail_id = 0
    method_of_largest_tail_id = None
    
    sps = softmax(-np.sort(-all_logits[batch_ind][ind_in_batch, time_point, :]))
    indices = np.argsort(-all_logits[batch_ind][ind_in_batch, time_point, :])
    ground_token = text[batch_ind][ind_in_batch][text_timepoint]
                
    for key,all_params in vals_dict.items():
        for par in all_params:
            method = key+'_'+str(par)
            if method in methods_wanted:
            
                if key == 'tfs':
                    first = sps[1:] - sps[:-1]
                    second = first[1:] - first[:-1]
                    tail_id = new_tfs(second, par)
                elif key=='flat':
                    tail_id = flat(sps, par)
                elif key=='n':
                    tail_id = nucleus_calc(sps, par)
                elif key=='k':
                    tail_id = par
                else:
                    print('key not recognized')
                    break
                    
                if tail_id > largest_tail_id:
                    largest_tail_id=tail_id
                    method_of_largest_tail_id=method
            
                diff_tail_positions[method] = tail_id#, ids_above_tail)

    if len(diff_tail_positions.keys()) >2: 
        print('CURRENTLY CANT LOOK AT MORE THAN 2 THINGS!')
        break
        
    # indices is still sorted. and can be used 
    smaller_method = list(diff_tail_positions.keys())
    smaller_method.remove(method_of_largest_tail_id)
    smaller_method = smaller_method[0]
    smaller_tail_id = diff_tail_positions[smaller_method]
    
    if largest_tail_id - smaller_tail_id < 3:
        print('not enough space between the tail locations')
        continue
        
    # what the ground token is and what the looser method is here. 
    ground_word = decoder_text([ground_token])
    
    #get the 15 words in front. 
    lead_prompt_words = decoder_text(text[batch_ind][ind_in_batch][text_timepoint-leading_prompt:text_timepoint])
    df_row = [prompt, time_point, batch_ind, ind_in_batch, lead_prompt_words, ground_word, method_of_largest_tail_id] # taking it from the last one. 
    
    for method in methods_wanted: # want to do them in the same order.
        # add the tail for each method: 
        df_row += [diff_tail_positions[method]]
        if method == method_of_largest_tail_id:
            unique_tokens = indices[smaller_tail_id:largest_tail_id]
            starting_pos = smaller_tail_id
        else:
            unique_tokens = indices[:smaller_tail_id]
            starting_pos = 0
            
        #remove any non word tokens: 
        unique_tokens = remove_non_words(unique_tokens)
        # select the right positions
        words_and_positions_wanted = get_specific_positions(unique_tokens, starting_pos)
        
        df_row += [words_and_positions_wanted[0], words_and_positions_wanted[1] ]
        
    to_df.append(df_row)
    
    
        

not enough space between the tail locations
not enough space between the tail locations
not enough space between the tail locations
not enough space between the tail locations
not enough space between the tail locations


In [176]:
'hey'.upper()

'HEY'

In [177]:
len(text[0][0][0:5])

5

In [178]:
results = pd.DataFrame(to_df, columns = ['prompt_ind', 'time_point', 
                               'batch_ind', 'ind_in_batch', 'leading_words',
                               'ground_word', 'looser_method', 'tail_pos_m1', 
        'specific_words_m1', 'positions_m1','tail_pos_m2','specific_words_m2', 'positions_m2'])

In [179]:
results.head()

,prompt_ind,time_point,batch_ind,ind_in_batch,leading_words,ground_word,looser_method,tail_pos_m1,specific_words_m1,positions_m1,tail_pos_m2,specific_words_m2,positions_m2
0,67,127,2,21,"hands together.`` My name is 28w35, this is m...",.'',tfs_0.95,7,"[None, None, None]","[0, 0, 0]",4,"[None, None, None]","[0, 0, 0]"
1,70,16,3,0,"\n onto Dirt, and still be mad \n at your doo...",you,tfs_0.95,8,"[your, it, they]","[3, 5, 8]",3,"[you, Dirt, she]","[0, 1, 3]"
2,33,127,1,10,"uses of technology. The infantry, what Hitosh...",to,n_0.69,32,"[the, now, mainly]","[0, 16, 32]",111,"[heavily, under, expected]","[32, 70, 109]"
3,21,66,0,21,"a bang, a bit like a firecracker, God appeare...",front,n_0.69,28,"[doorway, car, background]","[0, 14, 28]",63,"[street, midst, top]","[28, 45, 63]"
4,17,97,0,17,She was gorgeous to him and always had been. ...,believe,tfs_0.95,27,"[see, describe, even]","[17, 22, 27]",17,"[believe, stop, find]","[0, 8, 17]"


In [181]:
results.iloc[3,4]

' a bang, a bit like a firecracker, God appeared in the'

In [111]:
up =='A'

True

In [21]:
from utils import *

seed = 27
prompt_length = 100
gs=dict()
methods_wanted = ['tfs_0.95', 'n_0.69']#, 'k_40']
additional_path = '774M_model'
first = True


# randomly select generations and positions in those generations. from the original prompts. 




for key, params in vals_dict.items():
    for par in params:
        if key+'_'+str(par) in methods_wanted:
            # loading in all the methods
            all_logits = pickle.load( gzip.open('gpt-2_output/all_logits_'+key+'-sampling-type_'+par+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts.pickle.gz', 'rb'))
            text = pickle.load( gzip.open('gpt-2_output/all_text_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts_'+str(seed)+'-seed_'+additional_path+'.pickle.gz', 'rb'))
            rand_selects = pickle.load( gzip.open('gpt-2_output/prompt_rand_selections_'+key+'-sampling-type_'+str(par)+'-sampling-param_100-word-prompts_150-gen-length_100-number-of-prompts_'+str(seed)+'-seed_'+additional_path+'.pickle.gz', 'rb'))

            # finding the tail locations. 
            
            
            
            
            generated_output = []
            for batch in text:
                for entry in batch:
                    generated_output.append(decoder_text(entry[prompt_length:]))
            col_name = 'Samp Type:'+key+' Param:'+str(par)
            gs[col_name] = generated_output
            
    if first==True:
        prompts_used = []
        for batch in text:
            for entry in batch:
                prompts_used.append(decoder_text(entry[0:prompt_length]))
        
        first=False
gs = pd.DataFrame(gs)

In [22]:
gs.shape

(100, 2)

In [23]:
gs.head()

,Samp Type:tfs Param:0.95,Samp Type:n Param:0.69
0,"pass, and your time is up. You are one of the...",", days, weeks, months, years. The life you liv..."
1,n't I make sure he doesn't come back? \n \n \n...,I try to tell it not to worry? \n \n \n This ...
2,a lamp. So he was forced to create one. \n \n...,himself. He wished for the boy's family.\nAnd...
3,"we have her. She's now with us.''\n ``Oh, it'...",are holding her hostage. We'll have her for r...
4,", was a complete fucking idiot, who could neve...",", didn ’ t quite fit the character, and didn ’..."


In [24]:
gs.iloc[0,0]

' pass, and your time is up. You are one of the last people to leave the park, and the last one to see the city. You take your last breath, and watch as the night closes in. \n You are in your house, and your room is dark. The lights are on, and your computer is on, but you are not even sure what time it is. You sit up, and see the light from the front door come on, and your heart leaps. \n You close your eyes as you hear the phone ring. You hear the voicemail. You pick it up, and the caller says "Hi, I\'m your brother. \xa0I\'m home. \xa0I\'m sorry for the long wait'

In [25]:
gs.iloc[3]

Samp Type:tfs Param:0.95     we have her. She's now with us.''\n ``Oh, it'...
Samp Type:n Param:0.69       are holding her hostage. We'll have her for r...
Name: 3, dtype: object

In [26]:
for ind, p in enumerate(prompts_used):
    if 'Stockholm syndrome, but the other way around' in p:
        print(ind)

3


# Mass generator.

In [27]:
import numpy as np


In [28]:
num_gens = 10
file_name_base = 'test_big'
answer_keys = []
with open(file_name_base + '_blind_output.txt', 'w') as file: 
    
    for ind in range(num_gens):
    
        select_a_random_prompt = np.random.choice(gs.shape[0])
        select_random_generation_methods = np.random.choice(range(gs.shape[1]), size =gs.shape[1], replace=False )
        #select_random_generation_methods = np.insert(select_random_generation_methods, 0, 0)
        print(select_a_random_prompt)
        select_random_generation_methods;
        
        answer_keys.append(gs.iloc[select_a_random_prompt,select_random_generation_methods].index.values)

        file.write('Prompt: \n \n')
        file.write(prompts_used[select_a_random_prompt])
        file.write('\n')
        file.write('================== \n')
        for ind, out in enumerate(gs.iloc[select_a_random_prompt,select_random_generation_methods].tolist()):
            file.write('Random Generation: '+str(ind)+' \n \n')
            file.write(out)
            print(out)

            file.write('\n')
            file.write('================== \n')
            if ind == gs.shape[1]-1: # dont write random generation at the end. 
                continue
                
        file.write('\n \n')
        file.write('=====================================================')
        file.write('\n \n')
        
with open(file_name_base + '_answers.txt', 'w') as file: 
    for ak in answer_keys:
        for ind, k in enumerate(ak):
            file.write(str(ind)+' : '+k+' \n')
        file.write('\n')
        file.write('=====================================================')
        file.write('\n')
pickle.dump(answer_keys,open(file_name_base + '_answers_list.pickle','wb'))

10
 so she can't even tell you her first name. It's like she's a mother's worst nightmare.
How to get rid of a mother's greatest asset? Step on a crack, break a mother's back  #proud to be a mother
You can get a new one, too.
When women say they're a "whore" they mean that they have no self-control. What they mean is that they're all about sex and they'll do anything to get it. But the women who say they're a whore are always talking about one or two things:
A) To get off
B) To make sure they're being watched
C) To have an orgasm
When you see a whore
 where she's constantly exhausted and never really has much time to sleep. A single tear in a mother's eyes is worth a lifetime of misery.
Posted by Mark Liberman at 10:26 AM<|endoftext|>It is the year 2049.

As the United States military and the CIA attempt to create a "New World Order," a terrorist organization known as the Valkyries is leading the charge. With the help of a mysterious woman, the Valkyries are leading a mysterious rebell

Some of the very tight ones are too close and degenerate into repeats. The prompts and stochasticity of a single generation are really hard to deal with though. it is also a lot to read in order to evaluate the quality. 
What is something I can evaluate algorithmically and where diverse beam search is outperformed? It may inherently need to be something that is long. 
Seems like there is definitely a sweet spot where dont want too many options but also dont want it to be degenerate. 

In [54]:
answer_keys

[array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object)]

In [55]:
# assessing:


alex_answers = [1, 0.5, 0.5, 0, 0, 0, 1, 0.5, 0, 1] # liked generation 0
alex_chose = []

for ind, a in enumerate(alex_answers):
    if a==0.5:
        continue
    elif a==1:
        want_key_ind = 0
    elif a==0:
        want_key_ind = 1
    chosen = answer_keys[ind][want_key_ind]
    alex_chose.append(chosen)

In [57]:
answer_keys

[array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object),
 array(['Samp Type:tfs Param:0.95', 'Samp Type:n Param:0.69'], dtype=object),
 array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object)]

In [56]:
alex_chose

['Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69',
 'Samp Type:n Param:0.69']

In [49]:
gs.iloc[select_a_random_prompt,select_random_generation_methods]

array(['Samp Type:n Param:0.69', 'Samp Type:tfs Param:0.95'], dtype=object)

In [ ]:
gs.iloc[select_a_random_prompt,0]

In [ ]:
choose_prompt =rand_selects[select_a_random_prompt]
choose_prompt

In [ ]:
prompts.head()

In [ ]:
print(prompts.iloc[choose_prompt].test_target)